In [33]:
import numpy as np
import librosa
import sounddevice as sd
from keras.models import load_model
from pathlib import Path

# Configuration for audio processing and model
CONFIG = {
    "model_path": Path("D:\Code\ProjectsPython\ML_TrainingGround\ML_Audio\model.h5"),
    "mappings": [
        "five",
        "four",
        "go",
        "no",
        "off",
        "on",
        "one",
        "stop",
        "three",
        "tree",
        "two",
        "wow",
        "yes",
        "_background_noise_"
    ],
    "num_mfcc": 13,
    "num_frames": 44,
    "num_channels": 1,
    "sample_rate": 22500,
    "n_fft": 2048,
    "hop_length": 512,  # 10% of the sample rate
    "confidence_threshold": 0.6,
    "transponse": True
}

class SoundClassificationService:
    
    _instance = None

    def __init__(self, config):
        """Initialize the service with the given configuration."""
        self.config = config
        self.model = load_model(config["model_path"])
        
    @classmethod
    def get_instance(cls, config=CONFIG):
        """Singleton method to get the instance of the class."""
        if cls._instance is None:
            cls._instance = cls(config)
        return cls._instance
    
    def preprocess(self, audio):
        """Process the audio data to match the model's input requirements."""
        MFCCs = librosa.feature.mfcc(y=audio, sr=self.config["sample_rate"], n_mfcc=self.config["num_mfcc"], n_fft=self.config["n_fft"], hop_length=self.config["hop_length"])
        # Adjust MFCCs to required frame size
        num_required_mfcc = self.config["num_frames"]
        pad_width = num_required_mfcc - MFCCs.shape[1]
        if pad_width < 0:
            MFCCs = MFCCs[:, :num_required_mfcc]
        else:
            MFCCs = np.pad(MFCCs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        if self.config["transponse"]:
            MFCCs = MFCCs.T
        return MFCCs

    def predict(self, MFCCs):
        """Make a prediction based on preprocessed MFCCs."""
        MFCCs = MFCCs[np.newaxis, ..., np.newaxis]
        predictions = self.model.predict(MFCCs)
        predicted_index = np.argmax(predictions)
        confidence = predictions[0][predicted_index]
        if confidence > self.config["confidence_threshold"]:
            return self.config["mappings"][predicted_index], confidence
        return None

    def listen_and_predict(self, duration=1, overlap=0.5):
        """Listen to live audio and make predictions."""
        buffer_length = int(self.config["sample_rate"] * duration)
        buffer = np.zeros(buffer_length)
        try:
            with sd.InputStream(samplerate=self.config["sample_rate"], channels=1) as stream:
                print("Listening... Press Ctrl+C to stop.")
                while True:
                    audio_chunk, _ = stream.read(int(self.config["sample_rate"] * overlap))
                    buffer = np.roll(buffer, -len(audio_chunk))
                    buffer[-len(audio_chunk):] = audio_chunk.flatten()
                    MFCCs = self.preprocess(buffer)
                    keyword = self.predict(MFCCs)
                    if keyword:
                        print(f"Predicted Keyword: {keyword[0]}, at: {keyword[1]*100}%")
        except KeyboardInterrupt:
            print("Stopped listening.")


In [34]:
def main():
    # Initialize and start the sound classification service
    scs = SoundClassificationService.get_instance()
    scs.listen_and_predict(duration=1)

if __name__ == "__main__":
    main()

Listening... Press Ctrl+C to stop.
1/1 [==============================] - 0s 81ms/step
Predicted Keyword: two, at: 86.86814308166504%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: one, at: 99.73708391189575%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: one, at: 99.9953031539917%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: two, at: 99.60293769836426%
1/1 [==============================] - 0s 18ms/step
Predicted Keyword: three, at: 80.98357915878296%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: three, at: 94.04984712600708%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: wow, at: 64.23261761665344%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: wow, at: 96.011483669281%
1/1 [==============================] - 0s 17ms/step
Predicted Keyword: wow, at: 61.563873291015625%
1/1 [==============================] - 0s 18ms/step
Predicted Keyword: f